In [7]:
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders.csv_loader import CSVLoader

import pandas as pd
import os

In [3]:
chat_model = ChatOpenAI(api_key=os.environ['OPENAI_API_KEY'])

In [4]:
columns = {
    'name': [
        'RCP_SNO', 'RCP_TTL', 'CKG_NM', 'RGTR_ID', 'RGTR_NM', 'INQ_CNT',
        'RCMM_CNT', 'SRAP_CNT', 'CKG_MTH_ACTO_NM', 'CKG_STA_ACTO_NM', 
        'CKG_MTRL_ACTO_NM', 'CKG_KND_ACTO_NM', 'CKG_IPDC', 'CKG_MTRL_CN',
        'CKG_INBUN_NM', 'CKG_DODF_NM', 'CKG_TIME_NM', 'FIRST_REG_DT'
    ],
    'describe': [
        '레시피일련번호', '레시피제목', '요리명', '등록자ID', '등록자명', '조회수',
        '추천수', '스크랩수', '요리방법별명', '요리상황별명', 
        '요리재료별명', '요리종류별명', '요리소개', '요리재료내용',
        '요리인분명', '요리난이도명', '요리시간명', '최초등록일시'
    ]
}

columns = pd.DataFrame(columns)

columns.describe

<bound method NDFrame.describe of                 name describe
0            RCP_SNO  레시피일련번호
1            RCP_TTL    레시피제목
2             CKG_NM      요리명
3            RGTR_ID    등록자ID
4            RGTR_NM     등록자명
5            INQ_CNT      조회수
6           RCMM_CNT      추천수
7           SRAP_CNT     스크랩수
8    CKG_MTH_ACTO_NM   요리방법별명
9    CKG_STA_ACTO_NM   요리상황별명
10  CKG_MTRL_ACTO_NM   요리재료별명
11   CKG_KND_ACTO_NM   요리종류별명
12          CKG_IPDC     요리소개
13       CKG_MTRL_CN   요리재료내용
14      CKG_INBUN_NM    요리인분명
15       CKG_DODF_NM   요리난이도명
16       CKG_TIME_NM    요리시간명
17      FIRST_REG_DT   최초등록일시>

In [5]:
df = pd.read_csv('./data/receipe_dataset.csv')
df.columns = ['레시피일련번호', '레시피제목', '요리명', '등록자ID', '등록자명', '조회수',
              '추천수', '스크랩수', '요리방법별명', '요리상황별명', '요리재료별명', 
              '요리종류별명', '요리소개', '요리재료내용', '요리인분명', '요리난이도명', '요리시간명', '최초등록일시']

df = df[['요리명', '요리방법별명', '요리상황별명', '요리재료별명', '요리종류별명', '요리소개', '요리재료내용', '요리인분명',
    '요리난이도명', '요리시간명']]

df.columns = ['DishName', 'CookingMethod', 'CookingOccasion', 'MainIngredients', 'CuisineType', 'DishDescription', 
              'IngredientDetails', 'ServingSize', 'DifficultyLevel', 'CookingTime']
df.head(3)

C:\Users\tjddn\AppData\Local\Temp\ipykernel_27736\1816373519.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/receipe_dataset.csv')


,DishName,CookingMethod,CookingOccasion,MainIngredients,CuisineType,DishDescription,IngredientDetails,ServingSize,DifficultyLevel,CookingTime
0,어묵김말이,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,2인분,초급,60분이내
1,두부새우전,부침,일상,해물류,밑반찬,꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부...,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...,3인분,초급,30분이내
2,알밥,굽기,일상,해물류,밥/죽/떡,간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠...,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...,2인분,초급,30분이내
